In [4]:
import json
import os
import sys
import matplotlib.pyplot as plt
from PIL import Image
import importlib  # <-- Добавляем эту библиотеку

# --- Явное добавление пути к модулям ---

# Определяем абсолютный путь к корневой папке проекта
# Это делает код независимым от того, где запущен ноутбук
project_root = os.path.abspath(os.path.join(os.getcwd()))

# Формируем путь к папке 'src'
src_path = os.path.join(project_root, 'src')

# Добавляем путь в sys.path, если его там еще нет
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# --- Импорт нашего модуля ---

# Пытаемся импортировать. Если не получается, даем понятную ошибку
try:
    import face_verifier
    # Перезагружаем модуль на случай, если вы вносили в него изменения
    importlib.reload(face_verifier)
    from face_verifier import FaceVerifier
except ModuleNotFoundError:
    print(f"ОШИБКА: Не удалось найти модуль 'face_verifier.py'.")
    print(f"Убедитесь, что он лежит по пути: {src_path}")
    # Останавливаем выполнение, если модуль не найден
    raise

print("Модуль 'face_verifier' успешно импортирован.")

# --- Константы ---
TEST_FOLDER = 'data/test_data/evaluation_set'
MODELS_FOLDER = 'models'

D:\Coding\ML_2025\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Модуль 'face_verifier' успешно импортирован.


In [5]:
# ==============================================================================
# ===                        ЦЕНТР УПРАВЛЕНИЯ ТЕСТОМ                         ===
# ==============================================================================
#
# Просто измените эту переменную, чтобы выбрать, кого тестировать.
# Возможные значения: "kolya", "seva", "bogdan"
#
PERSON_TO_TEST = "seva"  # <-- ИЗМЕНИТЕ ЭТО ЗНАЧЕНИЕ ДЛЯ ДЕМОНСТРАЦИИ

# --- Конфигурация для каждого участника ---
TEST_SUBJECTS = {
    "kolya": {
        "ref_image": "data/test_data/kolya_reference_photo.jpg",
        "name_prefix": "kolya_"
    },
    "seva": {
        "ref_image": "data/test_data/seva_reference_photo.jpg",
        "name_prefix": "seva_"
    },
    "bogdan": {
        "ref_image": "data/test_data/bogdan_reference_photo.jpg",
        "name_prefix": "bogdan_"
    }
}

In [6]:
# Загружаем наш верификатор
verifier = FaceVerifier()

# Загружаем найденный порог
threshold_path = os.path.join(MODELS_FOLDER, 'verification_threshold.json')
try:
    with open(threshold_path, 'r') as f:
        data = json.load(f)
        threshold = data['best_threshold']
except FileNotFoundError:
    print(f"ОШИБКА: Файл с порогом '{threshold_path}' не найден! Запустите 'find_threshold.py' сначала.")
    threshold = None # Устанавливаем в None, чтобы дальнейший код не выполнился

if threshold:
    print(f"Верификатор загружен. Используется порог: {threshold:.4f}")

Используется устройство: cpu
ОШИБКА: Файл с порогом 'models\verification_threshold.json' не найден! Запустите 'find_threshold.py' сначала.


In [7]:
def visualize_verification(ref_image_path, test_image_path, distance, threshold, true_match):
    """
    Отрисовывает два изображения и выносит вердикт.
    """
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Загружаем изображения
    ref_img = Image.open(ref_image_path)
    test_img = Image.open(test_image_path)

    # Отображаем эталонное фото
    axes[0].imshow(ref_img)
    axes[0].set_title("Эталонное фото")
    axes[0].axis('off')

    # Отображаем тестовое фото
    axes[1].imshow(test_img)
    axes[1].set_title(f"Тестовое фото:\n{os.path.basename(test_image_path)}")
    axes[1].axis('off')

    plt.tight_layout()

    # Логика предсказания
    prediction_match = distance < threshold

    # Формируем вердикт
    verdict_text = "Совпадение (Вы)" if prediction_match else "Несовпадение (Не вы)"
    is_correct = (prediction_match == true_match)

    # Определяем цвет для текста вердикта
    color = 'green' if is_correct else 'red'

    # Выводим текстовую информацию под графиком
    print(f"Расстояние: {distance:.4f} (Порог: {threshold:.4f})")
    print(f"Вердикт модели: {verdict_text}")
    print(f"Правильный ответ: {'Совпадение' if true_match else 'Несовпадение'}")
    print(f"Результат: ", end="")

    # Печатаем цветной результат
    if is_correct:
        print("\033[92mПРАВИЛЬНО\033[0m") # Зеленый цвет
    else:
        print("\033[91mНЕПРАВИЛЬНО\033[0m") # Красный цвет

    plt.show()

In [8]:
if threshold:
    # Выбираем конфиг для нужного человека
    subject_config = TEST_SUBJECTS[PERSON_TO_TEST]
    ref_image_path = subject_config["ref_image"]
    name_prefix = subject_config["name_prefix"]

    print("="*60)
    print(f"=== ДЕМОНСТРАЦИЯ ДЛЯ: {PERSON_TO_TEST.upper()} ===")
    print(f"Эталонное фото: {ref_image_path}")
    print("="*60, "\n")

    # Получаем эмбеддинг для эталонного фото
    ref_embedding = verifier.get_embedding(ref_image_path)

    if ref_embedding is None:
        print("КРИТИЧЕСКАЯ ОШИБКА: Не удалось найти лицо на эталонном фото.")
    else:
        # Проходим по всем файлам в тестовой папке
        test_files = sorted([f for f in os.listdir(TEST_FOLDER) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

        for filename in test_files:
            test_image_path = os.path.join(TEST_FOLDER, filename)

            # Получаем эмбеддинг для тестового фото
            test_embedding = verifier.get_embedding(test_image_path)

            # Вычисляем расстояние
            distance = verifier.calculate_distance(ref_embedding, test_embedding)

            # Определяем, является ли фото на самом деле "своим"
            is_true_match = filename.startswith(name_prefix)

            print("-" * 50)

            # Визуализируем результат
            visualize_verification(ref_image_path, test_image_path, distance, threshold, is_true_match)

            print("\n")
else:
    print("Невозможно запустить демонстрацию, так как порог не был загружен.")

Невозможно запустить демонстрацию, так как порог не был загружен.
